In [ ]:
import os
import pandas as pd 

from app.podcasts.application import VespaApp

from vespa.io import VespaQueryResponse
with open("app/token.txt", "r") as f:
    token =  f.read()
os.environ['VESPA_CLOUD_SECRET_TOKEN'] = token

app = VespaApp(deploy=False, token=token)

In [ ]:
df = pd.read_csv("app/podcasts/data/transcribed_podcasts.csv")
resumos = pd.read_csv("full_data.csv", sep=";")

#join df and resumos on index
df = df.join(resumos, how="inner")
df = df.reset_index()
df.head(10)

In [ ]:
for i in range(0, len(df)):
    if 'Sharon' in df['response'][i]:
        print(i)

# sharon substitution
df.loc[157, "response"]= '    1. There\'s a subset of families in the US where children imitate their mothers all the time, even into adulthood.  2. Jonnafer\'s mother, Shirley Glass, is a psychologist in Baltimore. She recalls the time when her father left when she was a child, and how she had to take care of her three children.   3. Jonnafer\'s mother would talk in a country accent, and Jonna would imitate her by saying ""Sharon, Sharon, Sharon, please , Sharon, Sharon, Sharon, I know you hear me talking to you, Sharon, Sharon, get up, Sharon, I\'m going" '

In [ ]:
from vespa.io import VespaQueryResponse
soma_m1 = 0
soma_m2 = 0
soma_m3 = 0
count = 0

###################################################################
#################           TESTE MV             ##################
###################################################################


with app.app.syncio(connections=1) as session:
    
    ################# SEMANTIC-FULL TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# SEMANTIC-FULL-MV TITLE #######################
    for i in range(0, len(df)):
        query = df["title"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# SEMANTIC-FULL RESUMO LLM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# SEMANTIC-FULL-MV RESUMO LLM #######################
    for i in range(0, len(df)):
        query = df["response"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# FUSION-FULL TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0  
    ########################################################


    ################# FUSION-FULL-MV TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    

    # ################# FUSION-FULL RESUMO LLM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# FUSION-FULL-MV RESUMO LMM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    

In [ ]:
#############################################################
################# TESTES VARIANDO MODELOS ####################
#############################################################



with app.app.syncio(connections=1) as session:
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    ################# BM25 FULL RESUMO LLM ##########################
    # for i in range(0, len(df)):
    #     query = df["response"][i]
    #     response = session.query(
    #         yql="select * from sources * where userQuery() ",
    #         query=query,
    #         ranking="bm25_full",
    #         timeout="3000ms",
    #         hits=10,
    #     )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()
    #     #for m1 we add 1 if in top10
    #     #for m2 we add 1 if in top1
    #     #for m3 we add the position in the ranking (if not in top10 we add 11)

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_bm25_full_resumo: {soma_m1/count}\n")
    #     f.write(f"hr1_bm25_full_resumo: {soma_m2/count}\n")
    #     f.write(f"mrr_bm25_full_resumo: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    #     f.write("\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0

    # ################# BM25 FULL TITLE #######################
    # for i in range(0, len(df)):
    #     query = df["title"][i]
    #     response = session.query(
    #         yql="select * from sources * where userQuery() ",
    #         query=query,
    #         ranking="bm25_full",
    #         timeout="3000ms",
    #         hits=10,
    #     )

    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()
    #     #for m1 we add 1 if in top10
    #     #for m2 we add 1 if in top1
    #     #for m3 we add the position in the ranking (if not in top10 we add 11)
        
    #     if 'id' in app.hits_as_df(response, ['id']).columns:
    #         res = app.hits_as_df(response, ['id'])['id'].astype(int).tolist()
    #         count += 1
    #         if df["episode"][i] in res:
    #             soma_m1 += 1
    #             if res.index(df["episode"][i]) == 0:
    #                 soma_m2 += 1
    #             soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #         else:
    #             soma_m3 += 1/11
    #     else:
    #         print(f"{i} has no id")
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_bm25_full_title: {soma_m1/count}\n")
    #     f.write(f"hr1_bm25_full_title: {soma_m2/count}\n")
    #     f.write(f"mrr_bm25_full_title: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    #     f.write("\n")
    
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0
    # ################# BM25 FULL DESCRIPTION ##########################
    # for i in range(0, len(df)):
    #     query = df["description"][i]
    #     response = session.query(
    #         yql="select * from sources * where userQuery() ",
    #         query=query,
    #         ranking="bm25_full",
    #         timeout="3000ms",
    #         hits=10,
    #     )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()
    #     #for m1 we add 1 if in top10
    #     #for m2 we add 1 if in top1
    #     #for m3 we add the position in the ranking (if not in top10 we add 11)

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_bm25_full_description: {soma_m1/count}\n")
    #     f.write(f"hr1_bm25_full_description: {soma_m2/count}\n")
    #     f.write(f"mrr_bm25_full_description: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    #     f.write("\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0

    # ################# FUSION-FULL DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
                hits=10,
        )
        if i == 0 or i // 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_description: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_description: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# FUSION-FULL-MV DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
                hits=10,
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_description: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_description: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    ################# SEMANTIC-FULL DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
                hits=10,
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_description: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_description: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# SEMANTIC-FULL-MV DESCRIPTION #######################
    for i in range(0, len(df)):
        query = df["description"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
                hits=10,
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_description: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_description: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

In [ ]:
#############################################################
################# TESTES VARIANDO BUSCA ####################
#############################################################

with app.app.syncio(connections=1) as session:
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    # # ################# FUSION-DESC RESUMO LLM ######################
    # for i in range(0, len(df)):
    #     query = df["response"][i]
    #     response = session.query(
    #             yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_title_description,q)) limit 10",
    #             query=query,
    #             ranking="fusion_title_description",
    #             body={"input.query(q)": f"embed({query})"},
    #             timeout="3000ms",
    #             hits=10,
    #     )
    #     print(i)
    #     assert response.is_successful()

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_fusion_title_description_resumo: {soma_m1/count}\n")
    #     f.write(f"hr1_fusion_title_description_resumo: {soma_m2/count}\n")
    #     f.write(f"mrr_fusion_title_description_resumo: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0
    # ########################################################
    
     ################ BM25 DESC RESUMO LLM ##########################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
            yql="select * from sources * where userQuery() ",
            query=query,
            ranking="bm25_title_description",
            timeout="3000ms",
            hits=10,
        )
        print(i)
        assert response.is_successful()
        #for m1 we add 1 if in top10
        #for m2 we add 1 if in top1
        #for m3 we add the position in the ranking (if not in top10 we add 11)

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_bm25_title_description_resumo: {soma_m1/count}\n")
        f.write(f"hr1_bm25_title_description_resumo: {soma_m2/count}\n")
        f.write(f"mrr_bm25_title_description_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0